In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson
from scipy.stats import chi2
from scipy.stats import t
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests
plt.style.use('seaborn-whitegrid')

## Compton Gamma Ray Observatory Data set
### Goal:
Check the assumption that the emission rate is constant

#### Data
- X: Sequential time interval *(of variable lengths)*
- Y: Number of gamma rays originating in a particular area in the sky

#### Model
The phenomenon of interest seems to be related to a Poisson process. Therefore, we will model the data as

$G_i \sim Poisson(\lambda_i t_i)$
where

- $\lambda_i$ : average rate of gamma rays in interval i
- $t_i$ : length in second of the time interval

#### Hypothesis
$H_0: \lambda_0 = \lambda_1 = ... = \lambda_{99}$

$H_A: \lambda_i \neq \lambda_j$ for some *i and j*

In [ ]:
# Import data
df = pd.read_csv('./data/gamma-ray.csv')
data = df.copy()
t_i = data.loc[:, 'seconds']
G_i = data.loc[:, 'count']

data.columns = ['t_i', 'G_i']
data['lambda_i'] = G_i / t_i

In [ ]:
data.head()

In [ ]:
plt.scatter(t_i, G_i)
plt.xlabel('Sequential time interval (seconds)')
plt.ylabel('Number of gamma rays (count)')

plt.show()

We compute the Maximum Likelihood Estimators (MLE) $\hat{\lambda}$ for the parameter $\lambda$:

| MLE | $H_0$ is true | $H_A$ is true |
| --- | ------------- | ------------- |
| $\hat{\lambda}$ | $\frac{ \sum_i{ \frac{G_i}{t_i}}} {\sum_i{t_i} }$ | $\frac{G_i}{t_i}$ |

In [ ]:
MLE_0 = np.sum(data.G_i) / np.sum(data.t_i)
MLE_A = data.G_i / data.t_i

print('MLE given H_0 is true: %.3g' % MLE_0)

In [ ]:
plt.plot(t_i, poisson.pmf(G_i, MLE_0), 'bo', ms=3)
plt.title('Poisson distribution with parameters $\lambda$ = %.3g' % MLE_0)
plt.xlabel('Sequential time interval (seconds)')
plt.ylabel('Number of gamma rays (count)')

plt.show()

In [ ]:
# Likelihood Ratio Test
L = -2 * np.log((np.product(poisson.pmf(data.G_i, MLE_0))) / (np.product(poisson.pmf(data.G_i, MLE_A))))
print('Likelihood Ratio Test L = {:.3f}'.format(L))
d = MLE_A.shape[0] - 1

significance_level = 0.05
R_region = chi2.ppf(1 - significance_level, d)
print('Rejection region: L(x) >= {:.3f}'.format(R_region))

p_value = chi2.sf(L, d)
print('p-value for L(x): {:.3f}'.format(p_value))

In [ ]:
# Visualize the chi^2 distribution and p-value
x = np.arange(chi2.ppf(0.001, d), chi2.ppf(0.999, d))
plt.plot(x, chi2.pdf(x, d))
plt.vlines(L, 0, chi2.pdf(L, d), colors='g', label='L={:.3f}'.format(L))
plt.fill_between(x, chi2.pdf(x, d), where=x>=R_region, color='red', label='Rejection region (level={})'.format(significance_level))
plt.title('Chi^2 distribution with {} degrees of freedom\np-value = {:.3f}'.format(d, p_value))

plt.legend()
plt.show()

Since the p-value is greater than our significance level (0.337 > 0.05), we fail to reject $H_0$.
This means that the data does not significantly (at a significance level 0.05) show that $\lambda_i \neq \lambda_j$ for some i, j and the emission rate appears to be constant.

## Molecular classification of cancer (Golub et al. (1999)) dataset
### Goal:
Discover how many genes can be used to differentiate tumor types (acute lymphoblastic leukemia (ALL) and acute myeloid leukemia (AML)).
To do that we will use:
- uncorrected p-values
- Holm-Bonferoni correction, and Benjamini-Hochberg correction

In [ ]:
# Import data set
## Expression levels for the 38 tumor mRNA samples (row=genes, columns=mRNA samples)
gene_expression = pd.read_csv('./data/golub_data/golub.csv')
tumor_class = pd.read_csv('./data/golub_data/golub_cl.csv') # 0 -> ALL, 1 -> AML

gene_expression.drop('Unnamed: 0', axis=1, inplace=True)
tumor_class.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
gene_expression.head()
# tumor_class.head()

In [ ]:
# We separate the dataset into two data sets:
# - One with the expression levels for genes across the ALL mRNA samples
# - One with the expression levels for genes across the AML mRNA samples
indicator = tumor_class['x'].values.astype(bool)
AML_genes = gene_expression.iloc[:,indicator]
ALL_genes = gene_expression.iloc[:, indicator==False]

In [ ]:
barX_ALL = np.asarray(np.mean(ALL_genes, axis=1))
barX_AML = np.asarray(np.mean(AML_genes, axis=1))

N_ALL = ALL_genes.shape[1]
N_AML = AML_genes.shape[1]

s2_ALL = np.var(np.asarray(ALL_genes), axis=1, ddof=1)
s2_AML = np.var(np.asarray(AML_genes), axis=1, ddof=1)

s2_barX_ALL = s2_ALL / N_ALL
s2_barX_AML = s2_AML / N_AML

#### Welch unequal variances t-test
The Welch unequal variances t-test is similar to the t-test.

$t_{Welch}=\frac{|\bar{X}_{ALL} - \bar{X}_{AML}|}{\sqrt{\frac{s^2_{ALL}}{N_{ALL}}+{\frac{s^2_{AML}}{N_{AML}}}}} \sim t_\nu$.

As we consider a two-sided test, we could either use the absolute value of the difference in means.

The degrees of freedom parameter is given by the Welch-Satterthwaite formula:
$\nu = \frac{(\frac{s^2_{ALL}}{N_{ALL}}+{\frac{s^2_{AML}}{N_{AML}}})^2}
{\frac{1}{\nu_{ALL}}(\frac{s^2_{ALL}}{N_{ALL}})^2 + \frac{1}{\nu_{AML}}(\frac{s^2_{AML}}{N_{AML}})^2}$

In [ ]:
# delta_barX = barX_ALL - barX_AML
delta_barX = np.abs(barX_ALL - barX_AML)
s_delta_barX = s2_barX_ALL + s2_barX_AML

t_Welch = delta_barX / np.sqrt(s_delta_barX)

nu_ALL = N_ALL - 1
nu_AML = N_AML - 1
nu = np.floor((s_delta_barX ** 2) / (((1 / nu_ALL) * s2_barX_ALL ** 2) + ((1 / nu_AML) * s2_barX_AML ** 2)))

In [ ]:
significance_level = 0.05  / 2  # We divide the significance level by two as we consider a two-sided test
R_region = t.ppf(1 - significance_level, nu)
p_values = t.sf(t_Welch, nu)

# If absolute value is NOT used for delta_barX
# significant_p_values = np.concatenate((p_value[np.where(p_value <= significance_level)[0]],
#                                       p_value[np.where(p_value >= 1 - significance_level)]))
#If absolute value is used for delta_barX
significant_p_values = (p_values[np.where(p_values <= significance_level)[0]])
significant_t_Welch = t_Welch[np.where(np.abs(t_Welch) >= R_region)[0]]

print('Number of significant genes: (uncorrected p-values)', len(significant_p_values))
print('Number of |t_Welch| <= (Rejection region)_i: {}\n'.format(len(significant_t_Welch)))
# print('significant p-values:\n', np.round(significant_p_values, 4))
# print('\nsignificant t_Welch:\n', np.round(significant_t_Welch, 4))

In [ ]:
# As the different gene expression have different means and variances, we cannot plot all the results in one graph.
i = 10
nu_significant = nu[np.where(np.abs(t_Welch) >= R_region)[0]]
R_significant = R_region[np.where(np.abs(t_Welch) >= R_region)[0]]

x = np.linspace(t.ppf(0.0001, nu_significant[i]), t.ppf(0.9999, nu_significant[i]), 100)
plt.plot(x, t.pdf(x, nu_significant[i]))
plt.plot(significant_t_Welch[i], t.pdf(significant_t_Welch[i], nu_significant[i]), 'ro', markersize=3)
plt.fill_between(x, t.pdf(x, nu_significant[i]), where=np.abs(x)>=R_significant[i], alpha=0.3, color='r', lw=1)

plt.title('p-value: {:.3f}'.format(significant_t_Welch[i]))
plt.show()

In [ ]:
significance_level = 0.05
# Welch t-test using scipy.stats.ttest_ind (equal_var must be False so that Welch t-test is performed)
t_Welch, p_values = ttest_ind(ALL_genes, AML_genes, axis=1, equal_var=False)
significant_p_values = np.concatenate((p_values[np.where(p_values <= significance_level)], p_values[np.where(p_values >= 1 - significance_level)]))
print('Number of significant p-values using ttest_ind:', len(significant_p_values))

We performed multiple hypothesis testing but we did not correct for the p-values used to determine if we should reject $H_0$ or fail to reject it.

We will now perform such corrections, first by using the Holm-Bonferroni correction and then the Benjamini-Hochberg correction.

_Holm-Bonferroni correction:_ Reject ${H_0}_i$ if $p_i \leq \frac{\alpha}{m}$ where m is the number of hypotheses performed.

_Benjamini-Hochberg correction:_
   1. Sort the p-values in increasing order
   2. Find the maximum k such that $p_k \leq \frac{k * \alpha}{m}$
   3. Reject all the ${H_0}_i$ where $i \leq k$

In [ ]:
### Using libraries
# Number of significantly associated genes using p-value corrections
## Holm-Bonferroni correction
HM_correction = multipletests(p_values, alpha=significance_level, method='holm')
significant_HM_p_values = t_Welch[HM_correction[0]]
print('Significant p-values using the Holm-Bonferroni correction: ', len(significant_HM_p_values))

## Benjamini-Hochberg correction
BH_correction = multipletests(p_values, alpha=significance_level, method='fdr_bh')
significant_BH_p_values = t_Welch[BH_correction[0]]
print('Significant p-values using the Benjamini-Hochberg correction: ', len(significant_BH_p_values))

In [ ]:
### Implementation
def holm_bonferroni_correction(uncorrected_p_values, significance_level):
    sorted_p_values = np.sort(uncorrected_p_values)

    m = sorted_p_values.shape[0]
    i = 0
    while sorted_p_values[i] <= significance_level / (m - i + 1):
        i += 1

    significant_HM_p_values = sorted_p_values[:i]

    return significant_HM_p_values

def benjamini_hochberg_correction(uncorrected_p_values, significance_level):
    sorted_p_values = np.sort(uncorrected_p_values)

    m = sorted_p_values.shape[0]
    i = m-1
    while sorted_p_values[i] > (i * significance_level) / m and i >= 0:
        i -= 1

    significant_BH_p_values = sorted_p_values[:i]
    return significant_BH_p_values

In [ ]:
significant_HM_p_values = holm_bonferroni_correction(p_values, significance_level)
print('Significant p-values using the Holm-Bonferroni correction: ', len(significant_HM_p_values))
# print(significant_HM_p_values)

significant_BH_p_values = benjamini_hochberg_correction(p_values, significance_level)
print('Significant p-values using the Benjamini-Hochberg correction: ', len(significant_BH_p_values))
# print(significant_BH_p_values)

## Ordinary Least Squares (OLS) and Gradient Descent Algorithms

In [ ]:
def add_intercept(X):
    return np.concatenate((np.ones_like(X[:, :1]), X), axis=1)

In [ ]:
# Import data
X = np.genfromtxt('./data/syn_X.csv', delimiter=',', dtype=float)
y = np.genfromtxt('./data/syn_y.csv', delimiter=',', dtype=float)

X = add_intercept(X)

In [ ]:
X.shape
# y.shape

In [ ]:
OLS_estimate = np.matmul(np.matmul(np.linalg.inv(np.matmul(X.T, X)), X.T), y)
print('beta_hat (OLS):', np.round(OLS_estimate, 3))

In [ ]:
def visualize_convergence(losses, parameters, nb_iterations, step_size):
    iterations = np.arange(nb_iterations + 1)

    fig, ax = plt.subplots(1, 2, sharex=True, figsize=(10, 5))
    title = 'final loss={:.3g}'
    ax[0].plot(iterations, losses, color='blue')
    ax[0].set_title(title.format(losses[-1]))

    for i in range(parameters.shape[1]):
        beta_i = parameters[:, i]
        ax[1].plot(iterations, beta_i, label='$\\beta_{}$'.format(i))

    ax[1].set_title('Convergence of parameters')

    fig.suptitle('Gradient Descent, Loss: Squared Loss\nStep size: {}\nConverged after {} iterations'.format(step_size, nb_iterations))
    plt.legend(frameon=True)
    plt.show()

In [ ]:
def loss(beta, X, y):
    '''
    Squared error (loss) function to minimize
    '''
    return np.sum((np.matmul(beta, X.T) - y) ** 2)

def gradient_loss(beta, X, y):
    '''
    Gradient of the squared error function to minimize
    '''
    return 2 * np.matmul(np.matmul(beta, X.T) - y, X)

def gradient_descent(X, y, step_size, precision):
    '''
    Perform the Gradient Descent algorithm with 'step_size' (eta).
    Convergence is considered reached when the change in loss is less than 'precision'
    :param X: (ndarray) Feature matrix
    :param y: (ndarray) Dependent variable
    :param step_size: (int) step size by which to update the parameters with the gradient of the loss
    :param precision: (float)
    :return: OLS parameters, number of iteration until convergence
    '''
    n_sample, n_features = X.shape
    w_t = np.zeros(n_features)

    losses = []
    parameters = []

    gradient_step = lambda w_t: w_t - step_size * gradient_loss(w_t, X, y)
    change_in_loss = lambda w_t, w_t_1: np.abs(loss(w_t, X, y) - loss(w_t_1, X, y))  ## difference between the loss at w_t and the loss at w_t+1
    # change_in_loss = lambda w_t: np.linalg.norm(gradient_f(w_t, X, y), 2)  ## squared norm of the gradient of the loss at w_t+1

    w_t_1 = gradient_step(w_t)

    losses.append(loss(w_t_1, X, y))
    parameters.append(w_t_1)

    nb_iterations = 0
    while change_in_loss(w_t, w_t_1) >= precision:
        w_t = w_t_1
        w_t_1 = gradient_step(w_t)

        nb_iterations += 1
        losses.append(loss(w_t_1, X, y))
        parameters.append(w_t_1)

    visualize_convergence(losses, np.asarray(parameters), nb_iterations, step_size)

    return (w_t_1, nb_iterations)

In [ ]:
def run_gradient_descent(X, y, step_size=None):
    precision = 10e-6

    if step_size is not None:
        OLS_parameters, nb_iterations = gradient_descent(X, y, step_size=step_size, precision=precision)
    else:
        step_size = np.arange(0.001, 0.01, 0.001)

        nb_iterations = []
        OLS_parameters = []
        for alpha in step_size:
            OLS_GD, nb_iter = gradient_descent(X, y, step_size=alpha, precision=precision)
            nb_iterations.append(nb_iter)
            OLS_parameters.append(OLS_GD)

    return OLS_parameters, nb_iterations, step_size

In [ ]:
def get_best_run(OLS_parameters, nb_iterations, step_size):
    min_iterations = np.argmin(nb_iterations)
    best_alpha = step_size[min_iterations]
    parameters_best_alpha = OLS_parameters[min_iterations]

    return best_alpha, parameters_best_alpha

In [ ]:
OLS_parameters, nb_iterations, step_size = run_gradient_descent(X, y)

best_alpha, parameters_best_alpha = get_best_run(OLS_parameters, nb_iterations, step_size)
print('best alpha:', best_alpha)
print('OLS: ', np.round(parameters_best_alpha, 3))

### General Motors data set - Study of the contribution of air pollution to mortality

In [ ]:
# Import data
df = pd.read_csv('./data/mortality.csv')
y = df.Mortality
X = df.loc[:, df.columns != y.name].select_dtypes(['float64', 'int64'])

In [ ]:
print(X.shape)
X.head()

In [ ]:
X_array = add_intercept(np.asarray(X))
OLS_parameters, nb_iterations, step_size = run_gradient_descent(X_array, y)

It seems that no matter the step_size used in our Gradient Descent algorithm, the algorithm will not converge.

**Why would that be ?**

We will try to find out by first visualizing the distribution of our parameters.

In [ ]:
X.head()

In [ ]:
normalize = True

def plot_distributions(normalize=False, log_transform=False):
    columns = X.columns

    fig, ax = plt.subplots(len(columns), 1, figsize=(5, 30))
    fig.subplots_adjust(hspace=1)

    for i, column in enumerate(columns):
        x_i = X.loc[:, column]
        barX = np.mean(x_i)
        s_x = np.var(x_i)

        if normalize is True:
            x_i = (x_i - barX) / np.sqrt(s_x)

        if log_transform is True:
            x_i = np.log(x_i)

        ax[i].hist(x_i)
        ax[i].set_title('X={}\nE[X]={:.3f}, var(X)={:.3f}'.format(column, barX, s_x))

In [ ]:
plot_distributions(normalize=False, log_transform=True)

In [ ]:
plot_distributions(normalize=False, log_transform=False)
# plt.hist(y, label='Mortality')

We will now try again the GD algorithm with the normalized data

In [ ]:
X_array = np.asarray(X)
X_normalized = (X_array - np.mean(X_array, axis=0)) / np.sqrt(np.var(X_array, axis=0))
y_normalized = (y - np.mean(y)) / np.sqrt(np.var(y))
X_array = add_intercept(np.asarray(X_normalized))

OLS_parameters, nb_iterations, step_size = run_gradient_descent(X_array, y_normalized)

best_alpha, parameters_best_alpha = get_best_run(OLS_parameters, nb_iterations, step_size)
print('best alpha:', best_alpha)
print('OLS: ', np.round(parameters_best_alpha, 3))

In [ ]:
X_array = add_intercept(X_normalized)
OLS_parameters, nb_iterations, step_size = run_gradient_descent(X_array, y_normalized, 0.004)

In [ ]:
np.set_printoptions(suppress=True)
print('OLS parameters: ', np.round(OLS_parameters, 3))
# for param in OLS_parameters:
#     print('%.3f' % param)

In [ ]:
OLS_estimate = np.matmul(np.matmul(np.linalg.inv(np.matmul(X_array.T, X_array)), X_array.T), y_normalized)
print('beta_hat (OLS):', np.round(OLS_estimate, 3))
